In [1]:
# ===================================================================
# 6. MAIN EXPERIMENT EXECUTION
# ===================================================================

from model_utils import load_and_prepare_data
def run_experiment():
    """Main function to run the entire experiment."""
    # --- Prepare data once ---
    X_train, X_test, Y_train, Y_test, scaler_Y = load_and_prepare_data(
        FILE_PATH, FEATURES, TARGETS, TIME_STEPS, TEST_SIZE
    )

    # --- Define models to train ---
    models = {
        "LSTM": build_lstm_model,
        "GRU": build_gru_model,
        "TCN": build_tcn_model
    }
    
    results = {}

    input_shape = (X_train.shape[1], X_train.shape[2])
    output_size = len(TARGETS)
    Y_test_original = scaler_Y.inverse_transform(Y_test)

    # --- Loop through each model to train and evaluate ---
    for name, build_func in models.items():
        print(f"\n{'='*20} Training {name} Model {'='*20}")
        model = build_func(input_shape, output_size)
        model.summary()

        model.fit(
            X_train, Y_train,
            epochs=EPOCHS,
            batch_size=BATCH_SIZE,
            validation_data=(X_test, Y_test),
            verbose=1
        )

        # Sauvegarder le modèle après l'entraînement
        import os
        os.makedirs("saved_models", exist_ok=True)
        if name in ["LSTM", "GRU"]:
            model.save(f"saved_models/{name}_model.h5")
            print(f"Modèle {name} sauvegardé dans saved_models/{name}_model.h5")
        elif name == "TCN":
            model.save_weights(f"saved_models/{name}.weights.h5")
            print(f"Poids du modèle TCN sauvegardés dans saved_models/{name}.weights.h5")

        Y_pred_scaled = model.predict(X_test)
        Y_pred = scaler_Y.inverse_transform(Y_pred_scaled)
        
        mse = mean_squared_error(Y_test_original, Y_pred)
        rmse = np.sqrt(mse)
        mae = mean_absolute_error(Y_test_original, Y_pred)
        r2 = r2_score(Y_test_original, Y_pred)
        
        results[name] = {
            "MSE": mse,
            "RMSE": rmse,
            "MAE": mae,
            "R2": r2
        }

        print(f"\n📊 Regression Metrics for {name}")
        print(f"MSE : {mse:.4f}")
        print(f"RMSE: {rmse:.4f}")
        print(f"MAE : {mae:.4f}")
        print(f"R²  : {r2:.4f}")
        
        # NEW FUNCTION CALL ADDED HERE
        # Plot the first curve from the test set for this model
        print(f"Displaying example curve for {name} model...")
        plot_example_curve(Y_test_original[0], Y_pred[0], name, TARGETS)

    # --- Print final results summary ---
    print(f"\n{'='*20} Final Results Summary {'='*20}")
    print(pd.DataFrame(results).T)

    # --- Plot final comparison ---
    print("\nDisplaying final comparison metrics...")
    plot_comparison_metrics(results)

if __name__ == "__main__":
    run_experiment()


2025-09-01 12:46:41.372159: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


model_utils.py - TensorFlow Version: 2.16.2
model_utils.py - Keras Version: 3.10.0


NameError: name 'FILE_PATH' is not defined